In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import io
import requests
import datetime
import ondemand
from scipy.stats import pearsonr
from pandas.io.common import EmptyDataError
from requests.exceptions import HTTPError
import time
import os
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
from multiprocessing import Pool as ThreadPool
from multiprocessing import Manager
class Generate_Leaders():
    def __init__(self): 
        self.pageview_change = dict()
        self.my_date = None
    def get_mkt_cap(self, symbol, start_date, end_date):
        file = pd.read_csv('Highest_change_total.csv')
        try:
            mktcap = float(file.loc[file['Symbol'] == symbol]['Market_Cap(billion)'])
        except:
            return
        return mktcap
    def parallel(self, item):
        try:
            pageview_price = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(item.lower()))
        except FileNotFoundError:
            print('{} has no pageview data!!!'.format(item))
            return
        if pageview_price.empty:
            return
        #print(datetime.datetime.now())
        temp_res = pageview_price.set_index(['Date'])
        try:
            temp_res = temp_res.loc[[self.my_date]]
            temp_res.reset_index(col_fill = 'Date', inplace = True)
        except:
            print('{} has no trading on {}!!!'.format(item, self.my_date))
            return
        if temp_res.loc[0, 'PageView'] < 10:
            print('{} pageview on {} less than 10, skip it.'.format(item, self.my_date))
            return
        temp_res.loc[0, 'Market_Cap(billion)'] = self.get_mkt_cap(item, self.my_date, self.my_date)
        if temp_res['Market_Cap(billion)'].isnull().values.any():
            pass
        else:
            temp_res['Market_Cap(billion)'] = temp_res['Market_Cap(billion)'].round(2)
        self.pageview_change[item] = temp_res 
    def highest_change(self, my_date):
        midcap = pd.read_csv('mid_cap_symbols.csv')
        symbols = list(midcap['Symbol'])
        self.my_date = my_date
        for i, item in enumerate(symbols):
            if i % 300 == 0:
                print('run well')
                print(i)
            self.parallel(item)

In [3]:
midcap = pd.read_csv('mid_cap_symbols.csv')
all_symbols = list(midcap['Symbol'])

In [4]:
#from 2018-01-01 to 2018-12-31
#start = '2018-01-01'
daterange = pd.date_range(start = '2018-01-01', end = '2018-12-31')
daterange = daterange.strftime('%Y-%m-%d')

In [6]:
#Generate highest change for each day
for i, date in enumerate(daterange):
    print(i)
    print(date)
    leader = Generate_Leaders()
    highest_change = leader.highest_change(date)
    to_combine = pd.DataFrame([])
    for item in all_symbols:
        try:
            temp = leader.pageview_change[item]
        except:
            continue
        to_combine = pd.concat([to_combine, temp], sort = False, ignore_index = True)
    if not os.path.exists(os.path.dirname('leaderlist {}/'.format(date))):
        os.makedirs(os.path.dirname('leaderlist {}/'.format(date)))
    if to_combine.empty:
        print('{} is not trading day!!!'.format(date))
        continue
    highest_change_window1 = to_combine.drop(['PageView_Change_in_Std_3'], axis = 1)
    highest_change_window1['Time_Window'] = 1
    highest_change_window1.sort_values(by = ['PageView_Change_in_Std_1'], ascending = False, inplace = True)
    highest_change_window3 = to_combine.drop(['PageView_Change_in_Std_1'], axis = 1)
    highest_change_window3['Time_Window'] = 3
    highest_change_window3.sort_values(by = ['PageView_Change_in_Std_3'], ascending = False, inplace = True)
    highest_change_window1.to_csv('leaderlist {}/mid_highest_change_window1.csv'.format(date), index = False)
    highest_change_window3.to_csv('leaderlist {}/mid_highest_change_window3.csv'.format(date), index = False)
    

0
2018-01-01
run well
0
NUS has no trading on 2018-01-01!!!
ZTO has no trading on 2018-01-01!!!
CVCO has no trading on 2018-01-01!!!
DKS has no trading on 2018-01-01!!!
OMAB has no trading on 2018-01-01!!!
JEF has no trading on 2018-01-01!!!
LIVN has no trading on 2018-01-01!!!
MOMO has no trading on 2018-01-01!!!
JJSF has no trading on 2018-01-01!!!
IRDM has no trading on 2018-01-01!!!
BAH has no trading on 2018-01-01!!!
STAY has no trading on 2018-01-01!!!
CAE has no trading on 2018-01-01!!!
UHAL has no trading on 2018-01-01!!!
ASND has no trading on 2018-01-01!!!
CVA has no trading on 2018-01-01!!!
GIII has no trading on 2018-01-01!!!
JAG has no trading on 2018-01-01!!!
MMYT has no trading on 2018-01-01!!!
MDGL has no trading on 2018-01-01!!!
GLNG has no trading on 2018-01-01!!!
VICI has no trading on 2018-01-01!!!
TPX has no trading on 2018-01-01!!!
ASH has no trading on 2018-01-01!!!
G has no trading on 2018-01-01!!!
CUZ has no trading on 2018-01-01!!!
TSRO has no trading on 2018-

JLL has no trading on 2018-01-01!!!
MGP has no trading on 2018-01-01!!!
DNP has no trading on 2018-01-01!!!
HOG has no trading on 2018-01-01!!!
BPY has no trading on 2018-01-01!!!
WNS has no trading on 2018-01-01!!!
GKOS has no trading on 2018-01-01!!!
OAK has no trading on 2018-01-01!!!
DRH has no trading on 2018-01-01!!!
ST has no trading on 2018-01-01!!!
HBI has no trading on 2018-01-01!!!
TER has no trading on 2018-01-01!!!
MAIN has no trading on 2018-01-01!!!
WWE has no trading on 2018-01-01!!!
UFPI has no trading on 2018-01-01!!!
FGEN has no trading on 2018-01-01!!!
HOME has no trading on 2018-01-01!!!
SWN has no trading on 2018-01-01!!!
ENDP has no trading on 2018-01-01!!!
CONE has no trading on 2018-01-01!!!
KIM has no trading on 2018-01-01!!!
LSTR has no trading on 2018-01-01!!!
run well
300
UGI has no trading on 2018-01-01!!!
KBR has no trading on 2018-01-01!!!
ACC has no trading on 2018-01-01!!!
GGB has no trading on 2018-01-01!!!
NOAH has no trading on 2018-01-01!!!
CVNA ha

BRX has no trading on 2018-01-01!!!
NCR has no trading on 2018-01-01!!!
CABO has no trading on 2018-01-01!!!
NICE has no trading on 2018-01-01!!!
JHX has no trading on 2018-01-01!!!
RH has no trading on 2018-01-01!!!
BRC has no trading on 2018-01-01!!!
SNH has no trading on 2018-01-01!!!
ONB has no trading on 2018-01-01!!!
USFD has no trading on 2018-01-01!!!
IBTX has no trading on 2018-01-01!!!
AL has no trading on 2018-01-01!!!
NBR has no trading on 2018-01-01!!!
AHL has no trading on 2018-01-01!!!
CSL has no trading on 2018-01-01!!!
CHE has no trading on 2018-01-01!!!
FEYE has no trading on 2018-01-01!!!
SM has no trading on 2018-01-01!!!
HIW has no trading on 2018-01-01!!!
SOGO has no trading on 2018-01-01!!!
ENTG has no trading on 2018-01-01!!!
IONS has no trading on 2018-01-01!!!
ONCE has no trading on 2018-01-01!!!
TWOU has no trading on 2018-01-01!!!
SBS has no trading on 2018-01-01!!!
AGO has no trading on 2018-01-01!!!
MORN has no trading on 2018-01-01!!!
LOPE has no trading 

SPB has no trading on 2018-01-01!!!
SF has no trading on 2018-01-01!!!
RS has no trading on 2018-01-01!!!
PRGO has no trading on 2018-01-01!!!
AIMC has no trading on 2018-01-01!!!
PNR has no trading on 2018-01-01!!!
CLNY has no trading on 2018-01-01!!!
LEXEA has no trading on 2018-01-01!!!
RHP has no trading on 2018-01-01!!!
SEM has no trading on 2018-01-01!!!
POR has no trading on 2018-01-01!!!
ENIA has no trading on 2018-01-01!!!
AFG has no trading on 2018-01-01!!!
OSB has no trading on 2018-01-01!!!
DORM has no trading on 2018-01-01!!!
SAGE has no trading on 2018-01-01!!!
DOX has no trading on 2018-01-01!!!
LMBS has no trading on 2018-01-01!!!
IBKC has no trading on 2018-01-01!!!
WTFC has no trading on 2018-01-01!!!
VTIP has no trading on 2018-01-01!!!
IRWD has no trading on 2018-01-01!!!
ISTB has no trading on 2018-01-01!!!
VEON has no trading on 2018-01-01!!!
QTEC has no trading on 2018-01-01!!!
CLDR has no trading on 2018-01-01!!!
JCOM has no trading on 2018-01-01!!!
SYNH has no 

ALSN has no trading on 2018-01-01!!!
ACAD has no trading on 2018-01-01!!!
ARLP has no trading on 2018-01-01!!!
RNG has no trading on 2018-01-01!!!
ROIC has no trading on 2018-01-01!!!
SHO has no trading on 2018-01-01!!!
HOMB has no trading on 2018-01-01!!!
PAM has no trading on 2018-01-01!!!
PINC has no trading on 2018-01-01!!!
CBU has no trading on 2018-01-01!!!
IPGP has no trading on 2018-01-01!!!
PAGP has no trading on 2018-01-01!!!
MEOH has no trading on 2018-01-01!!!
RE has no trading on 2018-01-01!!!
FSLR has no trading on 2018-01-01!!!
ERIE has no trading on 2018-01-01!!!
WAL has no trading on 2018-01-01!!!
MANH has no trading on 2018-01-01!!!
KNX has no trading on 2018-01-01!!!
AERI has no trading on 2018-01-01!!!
PEGA has no trading on 2018-01-01!!!
IEI has no trading on 2018-01-01!!!
OFC has no trading on 2018-01-01!!!
WU has no trading on 2018-01-01!!!
APPN has no trading on 2018-01-01!!!
APO has no trading on 2018-01-01!!!
GGG has no trading on 2018-01-01!!!
PRI has no trad

...

HUBS pageview on 2018-01-04 less than 10, skip it.
STL pageview on 2018-01-04 less than 10, skip it.
BCPC pageview on 2018-01-04 less than 10, skip it.
IJT pageview on 2018-01-04 less than 10, skip it.
VET pageview on 2018-01-04 less than 10, skip it.
HQY pageview on 2018-01-04 less than 10, skip it.
QDEL pageview on 2018-01-04 less than 10, skip it.
ITCB pageview on 2018-01-04 less than 10, skip it.
NGHC pageview on 2018-01-04 less than 10, skip it.
ALTR pageview on 2018-01-04 less than 10, skip it.
TKC pageview on 2018-01-04 less than 10, skip it.
IGT pageview on 2018-01-04 less than 10, skip it.
ELY pageview on 2018-01-04 less than 10, skip it.
CRS pageview on 2018-01-04 less than 10, skip it.
JW.A pageview on 2018-01-04 less than 10, skip it.
CIT pageview on 2018-01-04 less than 10, skip it.
CISN pageview on 2018-01-04 less than 10, skip it.
BSTI pageview on 2018-01-04 less than 10, skip it.
IBOC pageview on 2018-01-04 less than 10, skip it.
STE pageview on 2018-01-04 less than 10,

AGO pageview on 2018-01-04 less than 10, skip it.
MORN pageview on 2018-01-04 less than 10, skip it.
LOPE pageview on 2018-01-04 less than 10, skip it.
TCP pageview on 2018-01-04 less than 10, skip it.
MUSA pageview on 2018-01-04 less than 10, skip it.
UMPQ pageview on 2018-01-04 less than 10, skip it.
AWR pageview on 2018-01-04 less than 10, skip it.
ELLI pageview on 2018-01-04 less than 10, skip it.
PRA pageview on 2018-01-04 less than 10, skip it.
TCF pageview on 2018-01-04 less than 10, skip it.
WRD pageview on 2018-01-04 less than 10, skip it.
KEX pageview on 2018-01-04 less than 10, skip it.
TSE pageview on 2018-01-04 less than 10, skip it.
MTX pageview on 2018-01-04 less than 10, skip it.
MBFI pageview on 2018-01-04 less than 10, skip it.
RYN pageview on 2018-01-04 less than 10, skip it.
REXR pageview on 2018-01-04 less than 10, skip it.
COMM pageview on 2018-01-04 less than 10, skip it.
FUL pageview on 2018-01-04 less than 10, skip it.
AMCX pageview on 2018-01-04 less than 10, 

CPA pageview on 2018-01-04 less than 10, skip it.
HLI pageview on 2018-01-04 less than 10, skip it.
VGSH pageview on 2018-01-04 less than 10, skip it.
PCTY pageview on 2018-01-04 less than 10, skip it.
FV pageview on 2018-01-04 less than 10, skip it.
FTSM pageview on 2018-01-04 less than 10, skip it.
VAC pageview on 2018-01-04 less than 10, skip it.
BXS pageview on 2018-01-04 less than 10, skip it.
HPT pageview on 2018-01-04 less than 10, skip it.
NWSA pageview on 2018-01-04 less than 10, skip it.
WBS pageview on 2018-01-04 less than 10, skip it.
PSXP pageview on 2018-01-04 less than 10, skip it.
PSO pageview on 2018-01-04 less than 10, skip it.
CNK pageview on 2018-01-04 less than 10, skip it.
VCLT pageview on 2018-01-04 less than 10, skip it.
MDU pageview on 2018-01-04 less than 10, skip it.
XT pageview on 2018-01-04 less than 10, skip it.
KFY pageview on 2018-01-04 less than 10, skip it.
TRCO pageview on 2018-01-04 less than 10, skip it.
NNI pageview on 2018-01-04 less than 10, skip

ADT has no trading on 2018-01-04!!!
APHA has no pageview data!!!
APY has no trading on 2018-01-04!!!
AVNS has no pageview data!!!
BH.A has no pageview data!!!
BHC has no pageview data!!!
CDAY has no pageview data!!!
CLNC has no pageview data!!!
COLD has no pageview data!!!
CWEN has no pageview data!!!
EAF has no pageview data!!!
GTES has no pageview data!!!
HUD has no pageview data!!!
HUYA has no pageview data!!!
JW.B has no trading on 2018-01-04!!!
LBRT has no pageview data!!!
MGY has no pageview data!!!
NVT has no pageview data!!!
PAGS has no pageview data!!!
PRSP has no pageview data!!!
PVTL has no pageview data!!!
RAMP has no pageview data!!!
ROAN has no pageview data!!!
SITC has no pageview data!!!
SMAR has no pageview data!!!
WH has no trading on 2018-01-04!!!
WHD has no pageview data!!!
WYND has no pageview data!!!
ZUO has no pageview data!!!
BILI has no pageview data!!!
DOCU has no pageview data!!!
GLIBA has no pageview data!!!
GSKY has no pageview data!!!
HWC has no pageview d

KeyboardInterrupt: 

In [102]:
#separate group for equities
for win in [1, 3]:
    for each in daterange1:
        try:
            highest_change = pd.read_csv('leaderlist {}/highest_change_window{}.csv'.format(each, win))
        except FileNotFoundError:
            continue
        small_cap_highest_change = highest_change.loc[highest_change['Market_Cap(billion)'] < 2]
        temp = highest_change.loc[highest_change['Market_Cap(billion)'] >= 2]
        mid_cap_highest_change = temp.loc[temp['Market_Cap(billion)'] < 10]
        large_cap_highest_change = temp.loc[temp['Market_Cap(billion)'] >= 10]
        if not os.path.exists(os.path.dirname('leaderlist {}/small_cap/window_{}_day/'.format(each, win))):
            os.makedirs(os.path.dirname('leaderlist {}/small_cap/window_{}_day/'.format(each, win)))
        if not os.path.exists(os.path.dirname('leaderlist {}/mid_cap/window_{}_day/'.format(each, win))):
            os.makedirs(os.path.dirname('leaderlist {}/mid_cap/window_{}_day/'.format(each, win)))
        if not os.path.exists(os.path.dirname('leaderlist {}/large_cap/window_{}_day/'.format(each, win))):
            os.makedirs(os.path.dirname('leaderlist {}/large_cap/window_{}_day/'.format(each, win)))
        mid_cap_highest_change.to_csv('leaderlist {}/mid_cap/window_{}_day/mid_cap_highest_change_time_window_{}.csv'.format(each, win, win), index = False)     
    

In [16]:
url_price = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=OnDemand&symbol={}&type=daily&startDate={}&endDate={}&maxRecords=1000&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration'

In [18]:
test = pd.read_csv(url_price.format('DM', '2018-12-01', '2018-12-30'))

In [19]:
test

,symbol,timestamp,tradingDay,open,high,low,close,volume,openInterest
0,DM,2018-12-03T00:00:00-05:00,2018-12-03,18.61,18.750,18.450,18.74,1671400,NaN
1,DM,2018-12-04T00:00:00-05:00,2018-12-04,18.74,19.020,18.660,18.75,1094600,NaN
2,DM,2018-12-06T00:00:00-05:00,2018-12-06,18.62,18.840,18.420,18.81,1653300,NaN
3,DM,2018-12-07T00:00:00-05:00,2018-12-07,18.74,19.210,18.740,19.05,1260300,NaN
4,DM,2018-12-10T00:00:00-05:00,2018-12-10,19.00,19.200,18.600,19.17,493000,NaN
5,DM,2018-12-11T00:00:00-05:00,2018-12-11,19.21,19.320,19.130,19.21,1379100,NaN
6,DM,2018-12-12T00:00:00-05:00,2018-12-12,19.25,19.280,18.940,18.97,898700,NaN
7,DM,2018-12-13T00:00:00-05:00,2018-12-13,18.94,19.230,18.940,19.14,3512700,NaN
8,DM,2018-12-14T00:00:00-05:00,2018-12-14,19.02,19.425,19.000,19.36,1333800,NaN
9,DM,2018-12-17T00:00:00-05:00,2018-12-17,19.19,19.420,18.540,18.65,2490700,NaN


Also, for "average pageview" please update this to = the 100 day average, not the average for the window.  For window 3, multiple the 100 day daily average by 3, and provide that value as the average.

Additional filters:

1.  Eliminate stocks with earnings release dates within +\- 5 days of the window end date.
2.  Expand the gap up/down filter to the past 10 days.
3.  Eliminate stocks with a 7% or greater percentage change (positive or negative) over the past 10 days from the window start date.
4.  Only include results for stocks with greater or equal to a 3 for actual pageviews in sd

In [4]:
#get business days
from pandas.tseries.offsets import *
daterange1 = pd.date_range('2017-12-01', '2019-01-20', freq=BDay())
daterange1 = list(daterange1.strftime('%Y-%m-%d'))
holidays = ['2017-12-25', '2018-01-01', '2018-01-15', '2018-02-19', '2018-03-30', '2018-05-28', '2018-07-04', '2018-09-03',\
            '2018-11-22', '2018-12-05', '2018-12-25', '2019-01-01']
for each in holidays:
    if each in daterange1:
        daterange1.remove(each)

In [5]:
daterange2 = daterange1[20:271]

In [13]:
#Pick the top 10 symbols with highest PageView_Change_in_std and PageView > 10
"""
Filters:
<1> Remove stocks with a gap up or down day within the window period
<2> Remove stocks that have a significantly greater high low range within the window period compared to normal.
"""
url_price = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=OnDemand&symbol={}&type=daily&startDate={}&endDate={}&maxRecords=1000&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration'
#get earning dates
def get_earning_date(symbol, start_date, end_date):
    url_earning = 'https://ondemand.websol.barchart.com/getCorporateActions.csv?apikey=ondemand&symbols={}&startDate={}&endDate={}&eventType=earnings&maxRecords=5'.\
                   format(symbol, start_date, end_date)
    try:
        return pd.read_csv(url_earning)
    except EmptyDataError as ede:
        #print(symbol + ' does not have earning_date during this period!!!')
        return pd.DataFrame([])
    except ConnectionResetError:
        print("getearning API problem")
        return get_earning_date(symbol, start_date, end_date)
def my_parallel(args):# filter for volatility.
    date = args[0]
    win_end = args[1]
    to_drop = args[2]
    window_days = args[3]
    gap_up_days = args[4]
    item = args[5]
    flag = True
    while flag:
        try:
            price = pd.read_csv(url_price.format(item, '2017-12-01', str(win_end)[:10]))
            flag = False
        except EmptyDataError:
            to_drop.append(item) # If the symbol has no trading for this window, drop it.
            print('EmptyData Error, {} has no trading on this window, drop it.'.format(item))
            flag = False
            return
        except ConnectionResetError:
            print('API Problem')
            continue
    price['high_loc_diff'] = price['high'] - price['low']
    price['H_L_20day_moving_avg'] = price['high_loc_diff'].rolling(20).mean()
    benchmark_index = price.loc[price['tradingDay'] == str(win_end)[:10]].index.item() - 1
    try:
        high_low_benchmark = price.iloc[benchmark_index, 10]
    except IndexError:
        print('Index Error')
        print(price)
    win_low = 0
    win_high = 0
    #filter for past 10 days percentage change > 7%
    win_start_index = daterange1.index(str(window_days[0])[:10])
    big_change_end = daterange1[win_start_index - 1]
    big_change_start = daterange1[win_start_index - 10]
    flag1 = True
    while flag1:  
        try:
            last_10_days_open = pd.read_csv(url_price.format(item, big_change_start, big_change_start))
            flag1 = False
        except EmptyDataError:
            to_drop.append(item)
            flag1 = False
            return
        except ConnectionResetError:
            print('API Problem')
            continue
    flag2 = True
    while flag2:
        try:
            last_10_days_close = pd.read_csv(url_price.format(item, big_change_end, big_change_end))
            flag2 = False
        except EmptyDataError:
            to_drop.append(item)
            flag2 = False
            return
        except ConnectionResetError:
            print('API Problem')
            continue
    if abs((last_10_days_close['close'][0] - last_10_days_open['open'][0])/last_10_days_open['open'][0]) > 0.07:
        print('prior 10 days open ' + str(last_10_days_open['open'][0]))
        print('close ' + str(last_10_days_close['close'][0]))
        print('change > 0.07, drop {}'.format(item))
        to_drop.append(item)
        return
    #filter for earning dates
    win_end_index = daterange1.index(str(window_days[-1])[:10])
    earning_start = daterange1[win_end_index - 5]
    earning_end = daterange1[win_end_index + 5]
    if get_earning_date(item, earning_start, earning_end).empty is False:
        print(item + ' has earning date, drop')
        to_drop.append(item)
        return
    for each in gap_up_days:#expand to the past 10 days.
        temp_price = price.loc[price['tradingDay'] == str(each)[:10]]
        try:
            last_day_index = temp_price.index.item() - 1
        except:
            print(temp_price)
        try:
            last_day_price = price.iloc[last_day_index]
        except IndexError:
            print('Index Error')
            print(price)
        try:
            if float(temp_price['open']) > float(last_day_price['high']) or\
            float(temp_price['open']) < float(last_day_price['low']):
                to_drop.append(item)
                return
        except TypeError:
            continue
    #processing window part
    for each in window_days:
        temp_price = price.loc[price['tradingDay'] == str(each)[:10]]
        try:
            last_day_index = temp_price.index.item() - 1
        except:
            print(temp_price)
        try:
            last_day_price = price.iloc[last_day_index]
        except IndexError:
            print('Index Error')
            print(price)
        try:
            if float(temp_price['open']) > float(last_day_price['high']) or\
            float(temp_price['open']) < float(last_day_price['low']):
                to_drop.append(item)
                return
        except TypeError:
            to_drop.append(item) # If the symbol has no trading for this window, drop it.
            print('TypeError, {} has no trading on this window, drop it.'.format(item))
            return
        if win_low == 0:
            win_low = float(temp_price['low'])
        else:
            win_low = min(float(temp_price['low']), win_low)
        if win_high == 0:
            win_high = float(temp_price['high'])
        else:
            win_high = max(float(temp_price['high']), win_high)
    win_high_low = float(win_high - win_low)
    if win_high_low > 2.5*high_low_benchmark:
        to_drop.append(item)
def my_filters(rawdata, date, win):
    symbols = list(rawdata['Symbol'])
    #temp_win_date = datetime.datetime.strptime(date, "%Y-%m-%d")
    count = 0
    window_days = []
    gap_up_days = []
    win_end_index = daterange1.index(date)
    while count < 10:
        if count < win:
            window_days.append(datetime.datetime.strptime(daterange1[win_end_index-count], "%Y-%m-%d"))
        else:
            gap_up_days.append(datetime.datetime.strptime(daterange1[win_end_index-count], "%Y-%m-%d"))
        count += 1
    window_days = window_days[::-1]
    gap_up_days = gap_up_days[::-1]
    win_start = window_days[0]
    win_end = window_days[-1]
    to_drop = Manager().list()
    args = []
    for item in symbols:
        args.append((date, win_end, to_drop, window_days, gap_up_days, item))
    pool = ThreadPool(10)
    pool.map(my_parallel, args) #important, no run except permission
    pool.close()
    pool.join() 

    to_drop = list(set(to_drop))
    for item in to_drop:
        rawdata = rawdata.loc[rawdata['Symbol'] != item]
    return rawdata
for win in [3]:# run win3
    for date in daterange2[47:]:
        print(datetime.datetime.now())
        print(date)
        for cap in ['mid']:
            try:
                to_filter = pd.read_csv\
                ('leaderlist {}/{}_cap/window_{}_day/{}_cap_highest_change_time_window_{}.csv'.format(date, cap, win, cap, win))
            except FileNotFoundError:
                continue
            to_filter = to_filter.loc[to_filter['PageView_Change_in_Std_{}'.format(win)] >= 3]
            to_filter = my_filters(to_filter, date, win)
            each = date
            each = each[5:]
            each = each.replace('-', '')
            to_filter = to_filter.head(10)
            to_filter.to_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win), index = False)
            

2019-03-18 16:34:11.943148
2018-03-12
prior 10 days open 154.5
close 168.06
change > 0.07, drop LGND
OKTA has earning date, drop
JBL has earning date, drop
prior 10 days open 41.92708
close 48.47509
change > 0.07, drop KFY
prior 10 days open 14.15
close 16.15
change > 0.07, drop EVH
prior 10 days open 19.26
close 30.21
change > 0.07, drop GTHX
prior 10 days open 27.3
close 37.5
prior 10 days open 47.57
change > 0.07, drop RGNX
close 53.95
change > 0.07, drop Z
TECD has earning date, drop
2019-03-18 16:34:15.396628
2018-03-13
prior 10 days open 43.14
close 47.61
change > 0.07, drop COUP
prior 10 days open 154.99
close 174.7
change > 0.07, drop LGND
prior 10 days open 14.2
close 15.75
change > 0.07, drop EVH
JBL has earning date, drop
prior 10 days open 68.65
close 83.25
change > 0.07, drop JOBS
DKS has earning date, drop
prior 10 days open 36.31
close 42.6
change > 0.07, drop OKTA
DSW has earning date, drop
GOLF has earning date, drop
prior 10 days open 42.23464
close 48.19729
change > 

close 21.23
change > 0.07, drop CLDR
prior 10 days open 12.7
close 10.8
change > 0.07, drop CZR
LW has earning date, drop
OLLI has earning date, drop
prior 10 days open 58.25
close 63.4
change > 0.07, drop CVLT
2019-03-18 16:35:09.810071
2018-04-06
prior 10 days open 19.54
close 22.24
change > 0.07, drop CLDR
prior 10 days open 82.05
close 91.05
change > 0.07, drop USNA
PSMT has earning date, drop
prior 10 days open 12.3
close 10.65
change > 0.07, drop CZR
LW has earning date, drop
OLLI has earning date, drop
2019-03-18 16:35:12.536848
2018-04-09
prior 10 days open 54.926230000000004
close 58.78382
change > 0.07, drop LW
prior 10 days open 82.05
close 92.65
change > 0.07, drop USNA
MSM has earning date, drop
PSMT has earning date, drop
2019-03-18 16:35:15.047882
2018-04-10
prior 10 days open 24.45
close 22.13
change > 0.07, drop ACAD
prior 10 days open 54.401990000000005
close 60.80164
change > 0.07, drop LW
MSM has earning date, drop
PSMT has earning date, drop
prior 10 days open 3.69

TCF has earning date, drop
LECO has earning date, drop
SKX has earning date, drop
CLF has earning date, drop
ALK has earning date, drop
WHR has earning date, drop
SC has earning date, drop
MDR has earning date, drop
prior 10 days open 42.16
close 45.85
change > 0.07, drop IONS
FFBC has earning date, drop
prior 10 days open 17.85
prior 10 days open 26.6
close 28.9
close 20.45
change > 0.07, drop TSG
change > 0.07, drop MNTA
API Problem
HOG has earning date, drop
MAN has earning date, drop
RNST has earning date, drop
prior 10 days open 41.67105
close 45.19571
change > 0.07, drop DK
prior 10 days open 64.03321
close 69.77445999999999
change > 0.07, drop BWXT
prior 10 days open 47.18
close 60.23
change > 0.07, drop AA
prior 10 days open 45.82
close 50.48
change > 0.07, drop ZEN
ONB has earning date, drop
EXPO has earning date, drop
FFIN has earning date, drop
prior 10 days open 69.968
close 77.25737
change > 0.07, drop NSP
ENTG has earning date, drop
JBLU has earning date, drop
GPK has ear

change > 0.07, drop WBS
HXL has earning date, drop
RHI has earning date, drop
CR has earning date, drop
prior 10 days open 106.2439
close 98.4186
change > 0.07, drop CCMP
prior 10 days open 95.65208
close 102.6944
prior 10 days open 17.6
change > 0.07, drop MORN
close 19.47
change > 0.07, drop ACAD
DRE has earning date, drop
prior 10 days open 20.53
close 24
change > 0.07, drop CRC
prior 10 days open 20.39
close 24.85
change > 0.07, drop WRD
ELY has earning date, drop
IPG has earning date, drop
EQM has earning date, drop
prior 10 days open 14.35
close 16.15
change > 0.07, drop STAA
IDA has earning date, drop
CUZ has earning date, drop
NTB has earning date, drop
prior 10 days open 56.0406
close 48.24635
change > 0.07, drop TEN
UCBI has earning date, drop
B has earning date, drop
GMED has earning date, drop
ST has earning date, drop
PFPT has earning date, drop
GT has earning date, drop
VVV has earning date, drop
prior 10 days open 24.839489999999998
close 26.770290000000003
change > 0.07

THC has earning date, drop
CCOI has earning date, drop
API Problem
prior 10 days open 203.2884
close 185.3593
change > 0.07, drop LFUS
INGN has earning date, drop
prior 10 days open 115.6781
close 107.266
change > 0.07, drop JBT
CUBE has earning date, drop
CCJ has earning date, drop
APPF has earning date, drop
IRWD has earning date, drop
prior 10 days open 33.78179
close 36.46117
change > 0.07, drop TRNO
MUSA has earning date, drop
prior 10 days open 50.96
close 58.4
change > 0.07, drop FCN
prior 10 days open 8.4
close 9.77
change > 0.07, drop I
IDA has earning date, drop
QTWO has earning date, drop
AGCO has earning date, drop
CZR has earning date, drop
TDOC has earning date, drop
VNOM has earning date, drop
prior 10 days open 140.1393
close 127.566
change > 0.07, drop CW
API Problem
ARCC has earning date, drop
prior 10 days open 39.73
close 36.47
change > 0.07, drop CRUS
ALRM has earning date, drop
NUS has earning date, drop
RYN has earning date, drop
SRC has earning date, drop
NCR ha

...

RBC has earning date, drop
NWL has earning date, drop
2019-03-18 16:43:05.681647
2018-08-08
API Problem
API Problem
prior 10 days open 37.75
close 44.58
change > 0.07, drop CARG
prior 10 days open 21.57
close 24.17
change > 0.07, drop MNK
prior 10 days open 24.25
close 28.9
change > 0.07, drop HMSY
prior 10 days open 29.82
close 26.93
change > 0.07, drop JELD
prior 10 days open 62.03
close 57.57
change > 0.07, drop Z
NWL has earning date, drop
SEAS has earning date, drop
TWLO has earning date, drop
prior 10 days open 116.9569
close 96.5166
change > 0.07, drop BLKB
prior 10 days open 208.4238
close 188.405
change > 0.07, drop MKTX
DAR has earning date, drop
TWO has earning date, drop
prior 10 days open 34.1
close 37.9
change > 0.07, drop STAA
OGE has earning date, drop
WU has earning date, drop
FIVN has earning date, drop
2019-03-18 16:43:08.579175
2018-08-09
prior 10 days open 38.9
close 44.68
change > 0.07, drop CARG
prior 10 days open 22.22
close 24.35
change > 0.07, drop MNK
YELP ha

THO has earning date, drop
2019-03-18 16:44:08.592614
2018-09-21
SCS has earning date, drop
2019-03-18 16:44:10.489639
2018-09-24
prior 10 days open 3.25
close 2.85
change > 0.07, drop AMRN
SCS has earning date, drop
2019-03-18 16:44:12.536747
2018-09-25
prior 10 days open 14.65675
close 15.886339999999999
change > 0.07, drop SCS
prior 10 days open 13.9
close 19.11
change > 0.07, drop ACAD
prior 10 days open 27.75
close 30.45
change > 0.07, drop UNVR
prior 10 days open 85.96
close 92.3
change > 0.07, drop ANAB
JBL has earning date, drop
prior 10 days open 3.19
close 2.93
change > 0.07, drop AMRN
prior 10 days open 61.42
close 68.42
change > 0.07, drop GHDX
2019-03-18 16:44:14.182105
2018-09-26
API Problem
prior 10 days open 27.72
close 31
change > 0.07, drop UNVR
prior 10 days open 62.87
close 69.15
change > 0.07, drop GHDX
JBL has earning date, drop
NEOG has earning date, drop
2019-03-18 16:44:16.205608
2018-09-27
API Problem
prior 10 days open 3.2
close 12.4
change > 0.07, drop AMRN


change > 0.07, drop MDR
change > 0.07, drop ACB
HE has earning date, drop
BKI has earning date, drop
MCY has earning date, drop
FLIR has earning date, drop
2019-03-18 16:45:09.676076
2018-11-01
API Problem
API Problem
API Problem
prior 10 days open 66.17596999999999
close 44.622840000000004
change > 0.07, drop CMP
prior 10 days open 1.93
close 1.61
change > 0.07, drop TRQ
prior 10 days open 15.9
close 13.15
change > 0.07, drop MDR
prior 10 days open 136.5811
close 150.8161
change > 0.07, drop PLCE
prior 10 days open 12.3
close 5.91
change > 0.07, drop ACB
prior 10 days open 36.20932
close 39.28461
NWSA has earning date, drop
change > 0.07, drop CVI
prior 10 days open 80.93555
close 72.80418
change > 0.07, drop ENS
HE has earning date, drop
2019-03-18 16:45:12.394352
2018-11-02
prior 10 days open 1.93
close 1.64
change > 0.07, drop TRQ
prior 10 days open 15.75
close 12.87
change > 0.07, drop MDR
HE has earning date, drop
prior 10 days open 10.0555
close 6.09
change > 0.07, drop ACB
prio

2019-03-18 16:45:59.562796
2018-11-26
2019-03-18 16:46:01.534287
2018-11-27
prior 10 days open 69.23917
close 64.00059
change > 0.07, drop CUB
prior 10 days open 45
close 38.77
change > 0.07, drop BL
2019-03-18 16:46:03.623827
2018-11-28
API Problem
prior 10 days open 42.66
close 38.85
change > 0.07, drop BL
prior 10 days open 24.019579999999998
prior 10 days open 57.45
close 25.70551
change > 0.07, drop PHM
close 50.73
change > 0.07, drop ENV
CBRL has earning date, drop
prior 10 days open 39.759370000000004
close 36.16928
change > 0.07, drop PBF
2019-03-18 16:46:06.320283
2018-11-29
prior 10 days open 23.43446
close 25.97328
change > 0.07, drop PHM
TECD has earning date, drop
CBRL has earning date, drop
API Problem
2019-03-18 16:46:09.348935
2018-11-30
API Problem
API Problem
prior 10 days open 53.24
close 58.76
change > 0.07, drop SAVE
TECD has earning date, drop
prior 10 days open 33.36785
close 28.15588
change > 0.07, drop BTU
2019-03-18 16:46:14.365588
2018-12-03
prior 10 days ope

prior 10 days open 62.06839
close 52.1486
change > 0.07, drop PRGO
prior 10 days open 3.131365
close 3.410595
change > 0.07, drop GFI
2019-03-18 16:46:57.475239
2018-12-27
prior 10 days open 22.59364
close 20.53078
change > 0.07, drop VET
prior 10 days open 71.68204
close 62.22834
change > 0.07, drop VAC
prior 10 days open 25.683239999999998
close 22.96995
change > 0.07, drop HPT
prior 10 days open 71.38
close 81.03
change > 0.07, drop PAC
2019-03-18 16:46:59.395043
2018-12-28
prior 10 days open 71.42
close 81.22
change > 0.07, drop PAC
prior 10 days open 14.239189999999999
close 12.91577
prior 10 days open 74.471
close 61.043890000000005
change > 0.07, drop DAN
prior 10 days open 18.39109
change > 0.07, drop VAC
close 16.48452
change > 0.07, drop MFGP
prior 10 days open 18.85146
close 16.52811
change > 0.07, drop SHLX
prior 10 days open 31.258290000000002
close 27.89278
change > 0.07, drop PWR
2019-03-18 16:47:02.035293
2018-12-31
API Problem
API Problem
API Problem
prior 10 days open

In [1]:
url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'

In [2]:
mytest = pd.read_csv(url.format('AAPL', '2018-03-20', '2018-03-20'))

NameError: name 'pd' is not defined

In [8]:
daterange2.index('2018-01-17')

10

In [26]:
#back test for 1-business day strategy
for win in [1, 3]:#run 3 day window
    for date in daterange2:
        print(date)
        for cap in ['mid']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = daterange1[daterange1.index(date) + 1]
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            invest = 0
            revenue = 0
            for item in portfolio:
                flag = True
                while flag:
                    try:
                        price = pd.read_csv(url.format(item, trading_day, trading_day))
                        flag = False
                    except ConnectionResetError:
                        print('API problem')
                        continue
                invest += price.loc[0, 'open']
                revenue += price.loc[0, 'close']
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            result = pd.DataFrame([], columns = ['Date', 'invest', 'hold(days)', 'portfolio_value', 'p&l', 'pct_change(%)', 'window'])
            if invest != 0:
                result.loc[0, ['Date', 'invest', 'hold(days)', 'portfolio_value',\
                           'p&l', 'pct_change(%)', 'window']] = [trading_day, invest, 1, revenue, revenue-invest, (revenue/invest - 1)*100, win]
            if not os.path.exists(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_1_day/'.format(date, cap, win))):
                os.makedirs(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_1_day/'.format(date, cap, win)))
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_1_day/{}_{}_cap_window_{}_hold_1_pl.csv'.format(date, cap, win, trading_each, cap, win), index = False)
            

2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11

In [28]:
#Generate trading detail for 1-business day
for win in [1, 3]:
    for date in daterange2:
        print(date)
        for cap in ['mid']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = daterange1[daterange1.index(date) + 1]
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            result = pd.DataFrame([], columns = ['symbol', 'win_start', 'win_end', 'actual_pageview', 'average_pageview', \
                                                 'std_100days_rolling', 'actual_pageview_in_std', 'buy_date', 'sell_date', \
                                                 'buy_price', 'sell_price', 'p&l', 'pct_change(%)', 'window'])
            count = 0
            window_days = []
            while count < win:
                window_days.append(daterange1[daterange1.index(date) - count])
                count += 1
            window_days = window_days[::-1]
            win_start = window_days[0]
            win_end = window_days[-1]
            result['symbol'] = portfolio
            result['win_start'] = str(win_start)[:10]
            result['win_end'] = str(win_end)[:10]
            result['buy_date'] = trading_day
            result['sell_date'] = trading_day
            for i, item in enumerate(portfolio):
                try:
                    price = pd.read_csv(url.format(item, trading_day, trading_day))
                except EmptyDataError:
                    result.loc[result['symbol'] == item, 'buy_price']= 'No Trading'
                    continue
                pageview_price = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(item))
                temp = pageview_price['PageView']
                pageview_price['std_100days_rolling'] = (temp.rolling(100).std()).shift(win)
                pageview_price['mean_100days_rolling'] = (temp.rolling(100).mean()).shift(win)
                total_pageview = 0
                for each in window_days:
                    total_pageview += int(pageview_price.loc[pageview_price['Date'] == str(each)[:10]]['PageView'])
                result.loc[i, 'actual_pageview'] = total_pageview
                result.loc[i, 'average_pageview'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['mean_100days_rolling'])*win
                result.loc[i, 'std_100days_rolling'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['std_100days_rolling'])
                result.loc[i, 'actual_pageview_in_std'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['PageView_Change_in_Std_{}'.format(win)])
                result.loc[i, 'buy_price'] = price.loc[0, 'open']
                result.loc[i, 'sell_price'] = price.loc[0, 'close']
                result.loc[i, 'p&l'] = price.loc[0, 'close'] - price.loc[0, 'open']
                result.loc[i, 'pct_change(%)'] = (price.loc[0, 'close']/price.loc[0, 'open'] - 1)*100
                result.loc[i, 'window'] = win
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_1_day/{}_{}_cap_window_{}_hold_1_detail.csv'.format(date, cap, win, trading_each, cap, win), index = False)
            
            
            

2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11

In [29]:
#to summary the 1-business day strategy results
for win in [1, 3]:
    for cap in ['mid']:
        to_combine = pd.DataFrame([])
        for date in daterange2:
            trading_day = daterange1[daterange1.index(date) + 1]
            each = trading_day
            each = each[5:]
            each = each.replace('-', '')
            try:
                temp = pd.read_csv('leaderlist {}/{}_cap/window_{}_day/hold_1_day/{}_{}_cap_window_{}_hold_1_pl.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                continue
            to_combine = pd.concat([to_combine, temp], sort = False, ignore_index = True)
        to_combine.loc['total_p&l', 'p&l'] = sum(to_combine.loc[to_combine['window'] == win]['p&l'])
        to_combine.loc['average_pct_change', 'pct_change(%)'] = to_combine['pct_change(%)'].mean()
        to_combine.loc['total_p&l', 'Date'] = 'Total_P&L'
        to_combine.loc['average_pct_change', 'Date'] = 'Average_Pct_Change'
        if not os.path.exists(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win))):
            os.makedirs(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win)))
        to_combine.to_csv('{}_cap/window_{}_day/{}_cap_window_{}_hold_1_pl_summary.csv'.format(cap, win, cap, win), index = False)    

In [30]:
#back test for 3-business day strategy
for win in [1, 3]:
    for date in daterange2:
        print(date)
        for cap in ['mid']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = daterange1[daterange1.index(date) + 1]
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            invest = 0
            revenue = 0
            for item in portfolio:
                buy_price = pd.read_csv(url.format(item, trading_day, trading_day))
                invest += buy_price.loc[0, 'open']
                sell_day = datetime.datetime.strptime(daterange1[daterange1.index(trading_day) + 2], "%Y-%m-%d")
                flag = True
                while flag:
                    try:
                        sell_price = pd.read_csv(url.format(item, sell_day, sell_day))
                        flag = False
                    except ConnectionResetError:
                        print('API problem')
                        continue
                revenue += sell_price.loc[0, 'close']
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            result = pd.DataFrame([], columns = ['Date', 'invest', 'hold(days)', 'portfolio_value', 'p&l', 'pct_change(%)', 'window'])
            if invest != 0:
                result.loc[0, ['Date', 'invest', 'hold(days)', 'portfolio_value',\
                           'p&l', 'pct_change(%)', 'window']] = [trading_day, invest, 3, revenue, revenue-invest, (revenue/invest - 1)*100, win]
            if not os.path.exists(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_3_day/'.format(date, cap, win))):
                os.makedirs(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_3_day/'.format(date, cap, win)))
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_3_day/{}_{}_cap_window_{}_hold_3_pl.csv'.format(date, cap, win, trading_each, cap, win), index = False)
        

2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11

In [11]:
url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
mytest = pd.read_csv(url.format('JW.B', '2018-06-08', '2018-06-12'))

In [31]:
#Generate trading detail for 3-business day
for win in [1, 3]:
    for date in daterange2:
        print(date)
        for cap in ['mid']:
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = daterange1[daterange1.index(date) + 1]
            portfolio = list(portfolio['Symbol'])
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            sell_day = datetime.datetime.strptime(daterange1[daterange1.index(trading_day) + 2], "%Y-%m-%d")
            result = pd.DataFrame([], columns = ['symbol', 'win_start', 'win_end', 'actual_pageview', 'average_pageview', \
                                                 'std_100days_rolling', 'actual_pageview_in_std', 'buy_date', 'sell_date', \
                                                 'buy_price', 'sell_price', 'p&l', 'pct_change(%)', 'window'])
            count = 0
            window_days = []
            while count < win:
                window_days.append(daterange1[daterange1.index(date) - count])
                count += 1
            window_days = window_days[::-1]
            win_start = window_days[0]
            win_end = window_days[-1]
            result['symbol'] = portfolio
            result['win_start'] = str(win_start)[:10]
            result['win_end'] = str(win_end)[:10]
            result['buy_date'] = trading_day
            result['sell_date'] = sell_day
            for i, item in enumerate(portfolio):
                try:
                    buy = pd.read_csv(url.format(item, trading_day, trading_day))
                except EmptyDataError:
                    result.loc[result['symbol'] == item, 'buy_price'] = 'No Trading'
                    continue
                try:
                    sell = pd.read_csv(url.format(item, sell_day, sell_day))
                except EmptyDataError:
                    result.loc[result['symbol'] == item, 'sell_price'] = 'No Trading'
                    continue
                pageview_price = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(item))
                temp = pageview_price['PageView']
                pageview_price['std_100days_rolling'] = (temp.rolling(100).std()).shift(win)
                pageview_price['mean_100days_rolling'] = (temp.rolling(100).mean()).shift(win)
                total_pageview = 0
                for each in window_days:
                    total_pageview += int(pageview_price.loc[pageview_price['Date'] == str(each)[:10]]['PageView'])
                result.loc[i, 'actual_pageview'] = total_pageview
                result.loc[i, 'average_pageview'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['mean_100days_rolling'])*win
                result.loc[i, 'std_100days_rolling'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['std_100days_rolling'])
                result.loc[i, 'actual_pageview_in_std'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['PageView_Change_in_Std_{}'.format(win)])
                result.loc[i, 'buy_price'] = buy.loc[0, 'open']
                result.loc[i, 'sell_price'] = sell.loc[0, 'close']
                result.loc[i, 'p&l'] = sell.loc[0, 'close'] - buy.loc[0, 'open']
                result.loc[i, 'pct_change(%)'] = (sell.loc[0, 'close']/buy.loc[0, 'open'] - 1)*100
                result.loc[i, 'window'] = win
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_3_day/{}_{}_cap_window_{}_hold_3_detail.csv'.format(date, cap, win, trading_each, cap, win), index = False)
            

2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11

In [32]:
#to summary the 3-business day strategy results
for win in [1, 3]:
    for cap in ['mid']:
        to_combine = pd.DataFrame([])
        for date in daterange2:
            trading_day = daterange1[daterange1.index(date) + 1]
            each = trading_day
            each = each[5:]
            each = each.replace('-', '')
            try:
                temp = pd.read_csv('leaderlist {}/{}_cap/window_{}_day/hold_3_day/{}_{}_cap_window_{}_hold_3_pl.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                continue
            to_combine = pd.concat([to_combine, temp], sort = False, ignore_index = True)   
        to_combine.loc['total_p&l', 'p&l'] = sum(to_combine.loc[to_combine['window'] == win]['p&l'])
        to_combine.loc['average_pct_change', 'pct_change(%)'] = to_combine['pct_change(%)'].mean()
        to_combine.loc['total_p&l', 'Date'] = 'Total_P&L'
        to_combine.loc['average_pct_change', 'Date'] = 'Average_Pct_Change'
        if not os.path.exists(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win))):
            os.makedirs(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win)))
        to_combine.to_csv('{}_cap/window_{}_day/{}_cap_window_{}_hold_3_pl_summary.csv'.format(cap, win, cap, win), index = False)         

In [33]:
#back test for buy close, hold one day, and sell close price tomorrow
for win in [1, 3]:
    for date in daterange2:
        print(date)
        for cap in ['mid']:
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = date
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            portfolio = list(portfolio['Symbol'])
            invest = 0
            portfolio_value = 0
            for item in portfolio:
                buy_price = pd.read_csv(url.format(item, trading_day, trading_day))
                invest += buy_price.loc[0, 'close']
                sell_day = daterange1[daterange1.index(trading_day) + 1]
                flag = True
                while flag:
                    try:
                        sell_price = pd.read_csv(url.format(item, sell_day, sell_day))
                        flag = False
                    except ConnectionResetError:
                        print('API Problem')
                        continue
                portfolio_value += sell_price.loc[0, 'close']
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            result = pd.DataFrame([], columns = ['Date', 'invest', 'hold(days)', 'portfolio_value', 'p&l', 'pct_change(%)', 'window'])
            if invest != 0:
                result.loc[0, ['Date', 'invest', 'hold(days)', 'portfolio_value',\
                           'p&l', 'pct_change(%)', 'window']] = [trading_day, invest, 1, portfolio_value, portfolio_value-invest, (portfolio_value/invest - 1)*100, win]
            if not os.path.exists(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/'.format(date, cap, win))):
                os.makedirs(os.path.dirname('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/'.format(date, cap, win)))
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/{}_{}_cap_window_{}_hold_1b_pl.csv'.format(date, cap, win, trading_each, cap, win), index = False)


2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11

In [35]:
#Generate trading detail for today close to next day close
for win in [1, 3]:
    for date in daterange2:
        print(date)
        for cap in ['mid']:
            each = date
            each = each[5:]
            each = each.replace('-', '')
            try:
                portfolio = pd.read_csv\
            ('leaderlist {}/{}_cap/window_{}_day/{}_{}_cap_window_{}_symbols.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                break
            url = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=onDemand&symbol={}&startDate={}&endDate={}&type=daily&splits=true&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ'
            trading_day = date
            portfolio = list(portfolio['Symbol'])
            trading_each = trading_day
            trading_each = trading_each[5:]
            trading_each = trading_each.replace('-', '')
            sell_day = daterange1[daterange1.index(trading_day) + 1]
            result = pd.DataFrame([], columns = ['symbol', 'win_start', 'win_end', 'actual_pageview', 'average_pageview', \
                                                 'std_100days_rolling', 'actual_pageview_in_std', 'buy_date', 'sell_date', \
                                                 'buy_price', 'sell_price', 'p&l', 'pct_change(%)', 'window'])
            temp_win_date = datetime.datetime.strptime(date, "%Y-%m-%d")
            window_days = []
            count = 0
            while count < win:
                window_days.append(daterange1[daterange1.index(date) - count])
                count += 1
            window_days = window_days[::-1]
            win_start = window_days[0]
            win_end = window_days[-1]
            result['symbol'] = portfolio
            result['win_start'] = str(win_start)[:10]
            result['win_end'] = str(win_end)[:10]
            result['buy_date'] = trading_day
            result['sell_date'] = sell_day
            for i, item in enumerate(portfolio):
                try:
                    buy = pd.read_csv(url.format(item, trading_day, trading_day))
                except EmptyDataError:
                    result.loc[result['symbol'] == item, 'buy_price'] == 'No Trading'
                    continue
                try:
                    sell = pd.read_csv(url.format(item, sell_day, sell_day))
                except EmptyDataError:
                    result.loc[result['symbol'] == item, 'sell_price'] == 'No Trading'
                    continue
                price = pd.read_csv(url.format(item, trading_day, sell_day))
                pageview_price = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(item))
                temp = pageview_price['PageView']
                pageview_price['std_100days_rolling'] = (temp.rolling(100).std()).shift(win)
                pageview_price['mean_100days_rolling'] = (temp.rolling(100).mean()).shift(win)
                total_pageview = 0
                for each in window_days:
                    total_pageview += int(pageview_price.loc[pageview_price['Date'] == str(each)[:10]]['PageView'])
                result.loc[i, 'actual_pageview'] = total_pageview
                result.loc[i, 'average_pageview'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['mean_100days_rolling'])*win
                result.loc[i, 'std_100days_rolling'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['std_100days_rolling'])
                result.loc[i, 'actual_pageview_in_std'] = float(pageview_price.loc[pageview_price['Date'] == str(win_end)[:10]]['PageView_Change_in_Std_{}'.format(win)])
                result.loc[i, 'buy_price'] = price.loc[0, 'close']
                result.loc[i, 'sell_price'] = price.loc[1, 'close']
                result.loc[i, 'p&l'] = price.loc[1, 'close'] - price.loc[0, 'close']
                result.loc[i, 'pct_change(%)'] = (price.loc[1, 'close']/price.loc[0, 'close'] - 1)*100
                result.loc[i, 'window'] = win
            result.to_csv('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/{}_{}_cap_window_{}_hold_1b_detail.csv'.format(date, cap, win, trading_each, cap, win), index = False)
            

2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-29
2018-01-30
2018-01-31
2018-02-01
2018-02-02
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-26
2018-02-27
2018-02-28
2018-03-01
2018-03-02
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11

In [36]:
#to summary the today's close to next close strategy results
for win in [1, 3]:
    for cap in ['mid']:
        to_combine = pd.DataFrame([])
        for date in daterange2:
            trading_day = date
            each = trading_day
            each = each[5:]
            each = each.replace('-', '')
            try:
                temp = pd.read_csv('leaderlist {}/{}_cap/window_{}_day/hold_1b_day/{}_{}_cap_window_{}_hold_1b_pl.csv'.format(date, cap, win, each, cap, win))
            except FileNotFoundError:
                continue
            to_combine = pd.concat([to_combine, temp], sort = False, ignore_index = True)   
        to_combine.loc['total_p&l', 'p&l'] = sum(to_combine.loc[to_combine['window'] == win]['p&l'])
        to_combine.loc['average_pct_change', 'pct_change(%)'] = to_combine['pct_change(%)'].mean()
        to_combine.loc['total_p&l', 'Date'] = 'Total_P&L'
        to_combine.loc['average_pct_change', 'Date'] = 'Average_Pct_Change'
        if not os.path.exists(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win))):
            os.makedirs(os.path.dirname('{}_cap/window_{}_day/'.format(cap, win)))
        to_combine.to_csv('{}_cap/window_{}_day/{}_cap_window_{}_hold_1b_pl_summary.csv'.format(cap, win, cap, win), index = False)
        
        